In [41]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

In [42]:
payload="{\"query\":\"query Chains {\\n    chains {\\n        id\\n    }\\n}\",\"variables\":{}}"

chains_response = requests.request("POST", url, headers=headers, data=payload)

chains_data = chains_response.json()

chains = chains_data['data']['chains']
chains = [c['id'] for c in chains]
chains


['eip155:5',
 'eip155:80001',
 'eip155:69',
 'eip155:11155111',
 'eip155:10',
 'eip155:421611',
 'eip155:43114',
 'eip155:84531',
 'eip155:56',
 'eip155:420',
 'eip155:1284',
 'eip155:137',
 'eip155:421613',
 'eip155:412346',
 'eip155:42170',
 'eip155:4',
 'eip155:97',
 'eip155:42',
 'eip155:534353',
 'eip155:1',
 'eip155:100',
 'eip155:42161',
 'eip155:43113']

In [43]:
import time
def get_proposal_vote_count_by_chain(chain_id: str):
    query = """
    query ProposalsWithVotes {
    proposals(chainId: "%s") {
        id
        voteStats {
        votes
        }
    }
    }""" % chain_id
    response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
    data = response.json()
    proposals = data['data']['proposals']
    # for each proposal, sum the votes
    for proposal in proposals:
        proposal['vote_count'] = sum([int(v['votes']) for v in proposal['voteStats']])
        # rm voteStats
        del proposal['voteStats']
    print('done', chain_id)
    # sleep 2s
    time.sleep(2)
    return proposals

In [44]:
proposal_vote_counts_by_chain = {}

In [45]:
# get_proposal_vote_count_by_chain(chains[0])
for chain in chains:
    if chain not in proposal_vote_counts_by_chain:
        proposal_vote_counts_by_chain[chain] = get_proposal_vote_count_by_chain(chain)


done eip155:5
done eip155:80001
done eip155:69
done eip155:11155111
done eip155:10
done eip155:421611
done eip155:43114
done eip155:84531
done eip155:56
done eip155:420
done eip155:1284
done eip155:137
done eip155:421613
done eip155:412346
done eip155:42170
done eip155:4
done eip155:97
done eip155:42
done eip155:534353
done eip155:1
done eip155:100
done eip155:42161
done eip155:43113


In [46]:
# save to json
with open('proposal_vote_counts_by_chain.json', 'w') as f:
    json.dump(proposal_vote_counts_by_chain, f)

In [47]:
# make a list of all proposals with a key for the chain
all_proposals = []
for chain in proposal_vote_counts_by_chain:
    for proposal in proposal_vote_counts_by_chain[chain]:
        proposal['chain'] = chain
        all_proposals.append(proposal)

all_proposals[:5]

[{'id': '20771534712125792765550139578267845622049543618311035993428633914963251436902',
  'vote_count': 1,
  'chain': 'eip155:5'},
 {'id': '1', 'vote_count': 0, 'chain': 'eip155:5'},
 {'id': '2', 'vote_count': 1, 'chain': 'eip155:5'},
 {'id': '58641171899228972540749114420077271382023318176672537994010128268707457712701',
  'vote_count': 0,
  'chain': 'eip155:5'},
 {'id': '3', 'vote_count': 0, 'chain': 'eip155:5'}]

In [48]:
import pandas as pd
df = pd.DataFrame(all_proposals)
df.head()

,id,vote_count,chain
0,2077153471212579276555013957826784562204954361...,1,eip155:5
1,1,0,eip155:5
2,2,1,eip155:5
3,5864117189922897254074911442007727138202331817...,0,eip155:5
4,3,0,eip155:5


In [49]:
df['vote_count'].describe([.99, .999, .9999])

count      7877.000000
mean         70.651898
std        1451.816916
min           0.000000
50%           1.000000
99%         201.240000
99.9%     26345.792000
99.99%    49323.099200
max       50117.000000
Name: vote_count, dtype: float64

In [55]:
# how many have 3000 or more votes?
big_proposals_df = df[df['vote_count'] >= 3000]
big_proposals_df

,id,vote_count,chain
3095,2787818427071270821149575583153491891613665380...,17893,eip155:10
3096,1147325722017097341143478593702267545197636573...,43200,eip155:10
3104,1036064007985958030126449663424034417437333554...,50117,eip155:10
3105,9436580542239877006792488137845550392842343963...,22913,eip155:10
3107,3297070190487044661440837301194291768042237675...,23124,eip155:10
3108,5173831469647334517214180804378233043006411761...,23615,eip155:10
3109,6493053874826825762192509371245455217377286098...,25669,eip155:10
3110,1036953249134245978023891813127229930376010326...,24225,eip155:10
7451,25,5836,eip155:1
7709,1107931230849807738961292165514452032072868489...,34833,eip155:42161


In [56]:
# save as big_proposals.csv
big_proposals_df.to_csv('over_3000_votes_proposals.csv', index=False)